<a href="https://colab.research.google.com/github/nascimento-luciano/big-data-analysis/blob/main/C%C3%B3pia_de_C%C3%B3pia_de_Language_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-p1q1h9zd
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-p1q1h9zd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 7.7 MB/s 
     |████████████████████████████████| 163 kB 71.6 MB/s 
  Created wheel for transformers: filename=transformers-4.24.0.dev0-py3-none-any.whl size=5256819 sha256=86d387aa152b186620f683c1021849343216af176619cbc5127089bfeb90fdd5
  Stored in directory: /tmp/pip-ephem-wheel-cache-dutr6ong/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
tokenizers                    0.13.1
transformers                  4.24.0.dev0


# Aplicando o BERT em uma Tarefa

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

example = "Oswaldo Goeldi nasceu no Brasil foi considerado o maior xilógrafo moderno."
ner_results = nlp(example)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Davlan--bert-base-multilingual-cased-ner-hrl/snapshots/6f69c39cadcdba0ab1401fb1f164964e7557e471/config.json
Model config BertConfig {
  "_name_or_path": "Davlan/bert-base-multilingual-cased-ner-hrl",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-DATE",
    "2": "I-DATE",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DATE": 1,
    "B-LOC": 7,
    "B-ORG": 5,
    "B-PER": 3,
    "I-DATE": 2,
    "I-LOC": 8,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_

In [ ]:
for element in ner_results:
  print(element['word'], element['entity'])

Oswald B-PER
##o I-PER
Go I-PER
##eld I-PER
##i I-PER
Brasil B-LOC


# Treinando um Modelo de Linguagem RoBERTa com o Hugging Face

## Baixando e Pré-processando o corpus

Baseado em:

*   https://towardsdatascience.com/pre-processing-a-wikipedia-dump-for-nlp-model-training-a-write-up-3b9176fdf67

* https://github.com/attardi/wikiextractor/

**Baixando o último dump da ptwiki**

In [ ]:
#!wget https://dumps.wikimedia.org/ptwiki/latest/ptwiki-latest-pages-articles.xml.bz2

--2022-10-10 16:23:49--  https://dumps.wikimedia.org/ptwiki/latest/ptwiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2020401966 (1.9G) [application/octet-stream]
Saving to: ‘ptwiki-latest-pages-articles.xml.bz2’

ptwiki-latest-pages 100%[===================>]   1.88G  4.78MB/s    in 6m 50s  

2022-10-10 16:30:40 (4.69 MB/s) - ‘ptwiki-latest-pages-articles.xml.bz2’ saved [2020401966/2020401966]



**Instalar o extrator**

In [ ]:
#!pip install wikiextractor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.9 MB/s 


**Baixando e executando o script de Extração e Limpeza do dump**

In [ ]:
#!wget https://gist.githubusercontent.com/sgraaf/7c061824b1c57c292faa0a123d95a714/raw/d635bc4c9e0094dd8103df87cf29a452d4491b07/extract_and_clean_wiki_dump.sh

--2022-10-10 16:30:52--  https://gist.githubusercontent.com/sgraaf/7c061824b1c57c292faa0a123d95a714/raw/d635bc4c9e0094dd8103df87cf29a452d4491b07/extract_and_clean_wiki_dump.sh
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 557 [text/plain]
Saving to: ‘extract_and_clean_wiki_dump.sh’

extract_and_clean_w 100%[===================>]     557  --.-KB/s    in 0s      

2022-10-10 16:30:52 (20.6 MB/s) - ‘extract_and_clean_wiki_dump.sh’ saved [557/557]



In [ ]:
############
# ~ 40 min #
############

#!sh /content/extract_and_clean_wiki_dump.sh ptwiki-latest-pages-articles.xml.bz2

Cloning into 'wikiextractor'...
remote: Enumerating objects: 766, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 766 (delta 13), reused 19 (delta 10), pack-reused 741
Receiving objects: 100% (766/766), 1.31 MiB | 827.00 KiB/s, done.
Resolving deltas: 100% (446/446), done.
Extracting and cleaning ptwiki-latest-pages-articles.xml.bz2 to ptwiki-latest-pages-articles.txt...
Succesfully extracted and cleaned ptwiki-latest-pages-articles.xml.bz2 to ptwiki-latest-pages-articles.txt


## Download da ptwiki processada

In [ ]:
################
###   FULL  ####
################

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BQq-3E70L7xRWOwf9nzzjHScILfaPKN1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BQq-3E70L7xRWOwf9nzzjHScILfaPKN1" -O ptwiki-latest-pages-articles.txt && rm -rf /tmp/cookies.txt

--2022-10-11 00:10:30--  https://docs.google.com/uc?export=download&confirm=t&id=1BQq-3E70L7xRWOwf9nzzjHScILfaPKN1
Resolving docs.google.com (docs.google.com)... 173.194.202.139, 173.194.202.138, 173.194.202.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.202.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-4k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vu0i1icbcpi6dvppbnjv6foboa8aokf0/1665447000000/17830184011945745179/*/1BQq-3E70L7xRWOwf9nzzjHScILfaPKN1?e=download&uuid=862015d8-0c5f-4760-8c33-ece7bfb26ee4 [following]
--2022-10-11 00:10:30--  https://doc-0g-4k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vu0i1icbcpi6dvppbnjv6foboa8aokf0/1665447000000/17830184011945745179/*/1BQq-3E70L7xRWOwf9nzzjHScILfaPKN1?e=download&uuid=862015d8-0c5f-4760-8c33-ece7bfb26ee4
Resolving doc-0g-4k-docs.googleusercontent.com (doc-0g-4k-docs.googleusercontent.com)... 74.125.195.1

In [ ]:
!head -100000 /content/ptwiki-latest-pages-articles.txt > train.txt

## Treinando o Tokenizador

In [ ]:
!mkdir /content/WikiBERT

In [ ]:
############
# ~ 32 min #
############

from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

#paths = [str(x) for x in Path("/content/").glob("**/*.txt")]
paths = ['/content/ptwiki-latest-pages-articles.txt']

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
tokenizer.save_model("WikiBERT")

['WikiBERT/vocab.json', 'WikiBERT/merges.txt']

**Download do Tokenizador Treinado**

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1M5vG9HIEUHd8N6A94gTFbQpSoihiP8v4' -O merges.txt

--2022-10-11 00:11:19--  https://docs.google.com/uc?export=download&id=1M5vG9HIEUHd8N6A94gTFbQpSoihiP8v4
Resolving docs.google.com (docs.google.com)... 172.253.117.101, 172.253.117.102, 172.253.117.138, ...
Connecting to docs.google.com (docs.google.com)|172.253.117.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-4k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kq6uhl1ulsrnkqm405v50936i2aav3r6/1665447075000/17830184011945745179/*/1M5vG9HIEUHd8N6A94gTFbQpSoihiP8v4?e=download&uuid=0325c160-e54b-4b90-8a14-12d3e4909718 [following]
--2022-10-11 00:11:20--  https://doc-04-4k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kq6uhl1ulsrnkqm405v50936i2aav3r6/1665447075000/17830184011945745179/*/1M5vG9HIEUHd8N6A94gTFbQpSoihiP8v4?e=download&uuid=0325c160-e54b-4b90-8a14-12d3e4909718
Resolving doc-04-4k-docs.googleusercontent.com (doc-04-4k-docs.googleusercontent.com)... 74.125.195.132, 2607:f

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1v1an01n8PLDozo3zWOA57FB_AbO6g6i8' -O vocab.json

--2022-10-11 00:11:23--  https://docs.google.com/uc?export=download&id=1v1an01n8PLDozo3zWOA57FB_AbO6g6i8
Resolving docs.google.com (docs.google.com)... 172.253.117.101, 172.253.117.102, 172.253.117.138, ...
Connecting to docs.google.com (docs.google.com)|172.253.117.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-4k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vmvdsbn4e7g7532vqd6adi64fihhrli8/1665447075000/17830184011945745179/*/1v1an01n8PLDozo3zWOA57FB_AbO6g6i8?e=download&uuid=adb82b4f-d457-45e5-9319-ba907d4cc088 [following]
--2022-10-11 00:11:23--  https://doc-08-4k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vmvdsbn4e7g7532vqd6adi64fihhrli8/1665447075000/17830184011945745179/*/1v1an01n8PLDozo3zWOA57FB_AbO6g6i8?e=download&uuid=adb82b4f-d457-45e5-9319-ba907d4cc088
Resolving doc-08-4k-docs.googleusercontent.com (doc-08-4k-docs.googleusercontent.com)... 74.125.195.132, 2607:f

**Movendo os arquivos de tokenização**

In [ ]:
!mv /content/merges.txt /content/WikiBERT/
!mv /content/vocab.json /content/WikiBERT/

**Testando o Tokenizador**

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "/content/WikiBERT/vocab.json",
    "/content/WikiBERT/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Meu nome é Joaquim!").tokens

['<s>', 'Meu', 'Ġnome', 'ĠÃ©', 'ĠJoaquim', '!', '</s>']

## Treinando o Modelo

In [ ]:
import torch

torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("WikiBERT", max_len=512)

**Passando as configurações para o Modelo**

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

**Tokenizando o arquivo de treino**

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="train.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

**Finalmente, treinar o modelo!**

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/WikiBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 100000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1563
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is p

Step,Training Loss
500,7.835400
1000,7.338200
1500,7.227100




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 30min 11s, sys: 5.72 s, total: 30min 17s
Wall time: 30min 17s


TrainOutput(global_step=1563, training_loss=7.456668397362844, metrics={'train_runtime': 1817.6151, 'train_samples_per_second': 55.017, 'train_steps_per_second': 0.86, 'total_flos': 3315678412800000.0, 'train_loss': 7.456668397362844, 'epoch': 1.0})

**Salvando o Modelo**

In [ ]:
trainer.save_model("/content/drive/MyDrive/Tutorial/WikiBERT/")

Saving model checkpoint to /content/drive/MyDrive/Tutorial/WikiBERT/
Configuration saved in /content/drive/MyDrive/Tutorial/WikiBERT/config.json
Model weights saved in /content/drive/MyDrive/Tutorial/WikiBERT/pytorch_model.bin


## Testando o Modelo Treinado

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1--9m2b0srAJl3gbkAlbiOmh_dAdX8PUW' -O config.json

--2022-10-11 01:01:53--  https://docs.google.com/uc?export=download&id=1--9m2b0srAJl3gbkAlbiOmh_dAdX8PUW
Resolving docs.google.com (docs.google.com)... 173.194.203.139, 173.194.203.101, 173.194.203.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.203.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6grucj0g2goaghu82hbhkefioo1759o5/1665450075000/17537572945470902047/*/1--9m2b0srAJl3gbkAlbiOmh_dAdX8PUW?e=download&uuid=c3e33cfd-d982-4ecc-9031-cedada93fe5f [following]
--2022-10-11 01:01:54--  https://doc-0c-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6grucj0g2goaghu82hbhkefioo1759o5/1665450075000/17537572945470902047/*/1--9m2b0srAJl3gbkAlbiOmh_dAdX8PUW?e=download&uuid=c3e33cfd-d982-4ecc-9031-cedada93fe5f
Resolving doc-0c-88-docs.googleusercontent.com (doc-0c-88-docs.googleusercontent.com)... 74.125.195.132, 2607:f

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-09h1HstcCKS30-77kxkzQFZY-SUN4gH' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-09h1HstcCKS30-77kxkzQFZY-SUN4gH" -O pytorch_model.bin && rm -rf /tmp/cookies.txt

--2022-10-11 01:03:48--  https://docs.google.com/uc?export=download&confirm=t&id=1-09h1HstcCKS30-77kxkzQFZY-SUN4gH
Resolving docs.google.com (docs.google.com)... 172.253.117.100, 172.253.117.102, 172.253.117.101, ...
Connecting to docs.google.com (docs.google.com)|172.253.117.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jqormllfiirbjdnsjjfvjuoo6nklkirp/1665450225000/17537572945470902047/*/1-09h1HstcCKS30-77kxkzQFZY-SUN4gH?e=download&uuid=0fe41757-05f0-40f5-bffe-f77ee2e011dd [following]
--2022-10-11 01:03:48--  https://doc-0o-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jqormllfiirbjdnsjjfvjuoo6nklkirp/1665450225000/17537572945470902047/*/1-09h1HstcCKS30-77kxkzQFZY-SUN4gH?e=download&uuid=0fe41757-05f0-40f5-bffe-f77ee2e011dd
Resolving doc-0o-88-docs.googleusercontent.com (doc-0o-88-docs.googleusercontent.com)... 74.125.195.1

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-6ycnq4LOBwnGDkCl5bT32KpaJ-haUnM' -O training_args.bin

--2022-10-11 01:05:24--  https://docs.google.com/uc?export=download&id=1-6ycnq4LOBwnGDkCl5bT32KpaJ-haUnM
Resolving docs.google.com (docs.google.com)... 173.194.202.113, 173.194.202.138, 173.194.202.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.202.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9ikn6c217af7khfdbbffrnscnmfdcj04/1665450300000/17537572945470902047/*/1-6ycnq4LOBwnGDkCl5bT32KpaJ-haUnM?e=download&uuid=0dcdbabc-5252-4d2d-b6a1-f660b63c02c0 [following]
--2022-10-11 01:05:24--  https://doc-0s-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9ikn6c217af7khfdbbffrnscnmfdcj04/1665450300000/17537572945470902047/*/1-6ycnq4LOBwnGDkCl5bT32KpaJ-haUnM?e=download&uuid=0dcdbabc-5252-4d2d-b6a1-f660b63c02c0
Resolving doc-0s-88-docs.googleusercontent.com (doc-0s-88-docs.googleusercontent.com)... 74.125.195.132, 2607:f

In [ ]:
!mv /content/config.json /content/WikiBERT
!mv /content/pytorch_model.bin /content/WikiBERT
!mv /content/training_args.bin /content/WikiBERT

**Adivinhando a próxima palavra**

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="/content/WikiBERT",
    tokenizer="/content/WikiBERT"
)

loading configuration file /content/WikiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/WikiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file /content/WikiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/WikiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 

In [ ]:
fill_mask("Mark Rothko nasceu em <mask>")

[{'score': 0.5601575970649719,
  'token': 18,
  'token_str': '.',
  'sequence': 'Mark Rothko nasceu em.'},
 {'score': 0.04723898693919182,
  'token': 30,
  'token_str': ':',
  'sequence': 'Mark Rothko nasceu em:'},
 {'score': 0.007757669314742088,
  'token': 354,
  'token_str': ' (',
  'sequence': 'Mark Rothko nasceu em ('},
 {'score': 0.005470750853419304,
  'token': 339,
  'token_str': ' "',
  'sequence': 'Mark Rothko nasceu em "'},
 {'score': 0.0053644790314137936,
  'token': 263,
  'token_str': ' de',
  'sequence': 'Mark Rothko nasceu em de'}]